In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
from datasets import load_dataset

dataset = load_dataset("xsum", cache_dir="/content/drive/MyDrive/")

In [ ]:
dataset = dataset["train"]

In [ ]:
texts = dataset["document"]

In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
ans = []



In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"


In [ ]:
model = model.to(device)


In [ ]:
@torch.no_grad()
def get_embs(texts, version):
  ans = []
  if (version != -1):
    ans = list(torch.load("/content/drive/MyDrive/file_train" +str(version)+".txt"))
    for i in range(len(ans)):
      ans[i] = ans[i].reshape(1, len(ans[i]))
  for i in range(version + 1, len(texts)):
   print(i)
   inputs = tokenizer(texts[i],  return_tensors="pt", truncation=True).to(device)
   x = model.forward(**inputs).encoder_last_hidden_state[0]
   tmp = []
   for j in range(len(x)):
     if inputs["attention_mask"][0][j]:
       tmp.append(x[j])
   ans.append(torch.mean(x, axis=0).cpu().reshape(1, len(x[0])))
   torch.cuda.empty_cache()
   if i % 10000 == 0:
     torch.save(torch.cat(ans), "/content/drive/MyDrive/file_train" +str(i)+".txt")
  return ans


In [ ]:
trains = get_embs(texts, -1)

In [ ]:
ans = torch.cat(ans)

In [ ]:
torch.save(trains, "/content/drive/MyDrive/file_train.txt")